In [83]:
import gensim
import numpy as np
import pandas as pd
import umap

import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (15.0, 8.0)
pylab.rcParams['figure.dpi'] = 80

import logging
logging.basicConfig(
    level=logging.DEBUG, 
    format='%(levelname)-8s [%(lineno)d] %(message)s')

DEBUG    [415] CACHEDIR=/home/makrai/.cache/matplotlib
DEBUG    [1362] Using fontManager instance from /home/makrai/.cache/matplotlib/fontlist-v300.json
DEBUG    [211] Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG    [211] Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG    [211] Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Populating the interactive namespace from numpy and matplotlib


# Load prevlex

In [ ]:
prevlex = pd.read_csv('../../prevlex/PrevLex.txt', sep='\t', header=None, index_col=0,
                      names=[#'prev_verb', 
                          'tokfreq', 'anal', 'docfreq', 'normalized'])

In [32]:
prevlex.sort_values(by='tokfreq', ascending=False).head()

,tokfreq,anal,docfreq,normalized,prev,verb,prevverb
el+mond,254053,FIN,1495,el+mond,el,mond,elmond
ki+derül,132541,FIN,1459,ki+derül,ki,derül,kiderül
meg+jelenik,113081,FIN,1538,meg+jelenik,meg,jelenik,megjelenik
el+fogad,111710,FIN,1424,el+fogad,el,fogad,elfogad
meg+van,94326,FIN,1506,meg+van,meg,van,megvan


In [36]:
prevlex[prevlex.index != prevlex.normalized].sort_values(by='tokfreq', ascending=False).head()

,tokfreq,anal,docfreq,normalized,prev,verb,prevverb
meg+hackel,134,UNKNOWN,73,meg+hekkel,meg,hekkel,meghekkel
be+copyzik,33,UNKNOWN,21,be+kopiz,be,kopiz,bekopiz
ide+copyzik,23,UNKNOWN,18,ide+kopiz,ide,kopiz,idekopiz
bele+applikál,19,FIN,18,bele+aplikál,bele,aplikál,beleaplikál
el+bagatelizál,18,UNKNOWN,16,el+bagatellizál,el,bagatellizál,elbagatellizál


In [31]:
for i, name in enumerate(['prev', 'verb']):
    prevlex[name] = prevlex.normalized.apply(lambda s: s.split('+')[i])
prevlex['prevverb'] = prevlex['prev'] + prevlex['verb']

# Vectors

In [3]:
embed = gensim.models.KeyedVectors.load(
    '/mnt/permanent/Language/Hungarian/Embed/mnsz2_webcorp/word2vec-mnsz2-webcorp_600_w10_n5_i1_m10.gensim')

In [134]:
invocab = prevlex[prevlex.verb.isin(embed.vocab) & prevlex.prevverb.isin(embed.vocab)]

In [114]:
def get_vector(word):
    if word in embed.vocab:
        return embed.syn0[embed.vocab[word].index]
    else:
        return np.nan

In [142]:
invocab = pd.concat([
    invocab, 
    (invocab.prevverb.apply(get_vector) - invocab.verb.apply(get_vector)).to_frame()])

In [143]:
invocab.head()

,0,anal,docfreq,normalized,prev,prevverb,tokfreq,vector,verb
abba+hagy,NaN,FIN,1258.0,abba+hagy,abba,abbahagy,9496.0,"[-0.032553323, -0.029644798, 0.0669592, 0.1045...",hagy
abba+marad,NaN,FIN,620.0,abba+marad,abba,abbamarad,1277.0,"[-0.08617115, -0.052307848, -0.020444728, 0.07...",marad
agyon+lő,NaN,FIN,605.0,agyon+lő,agyon,agyonlő,2761.0,"[0.037195563, -0.07621741, -0.038697153, 0.008...",lő
agyon+ver,NaN,FIN,659.0,agyon+ver,agyon,agyonver,1783.0,"[0.05801889, -0.05379776, 0.040726796, 0.04713...",ver
agyon+üt,NaN,FIN,359.0,agyon+üt,agyon,agyonüt,613.0,"[0.06812144, -0.004928887, 0.038111962, -0.067...",üt


In [ ]:
mapping = umap.UMAP(metric='cosine')

In [154]:
invocab.vector

abba+hagy            [-0.032553323, -0.029644798, 0.0669592, 0.1045...
abba+marad           [-0.08617115, -0.052307848, -0.020444728, 0.07...
agyon+lő             [0.037195563, -0.07621741, -0.038697153, 0.008...
agyon+ver            [0.05801889, -0.05379776, 0.040726796, 0.04713...
agyon+üt             [0.06812144, -0.004928887, 0.038111962, -0.067...
agyon+csap           [-0.078961045, 0.030715792, 0.08820374, 0.0113...
agyon+nyom           [-0.100824684, -0.070471086, 0.017328229, 0.03...
agyon+tapos          [0.03918902, -0.040448576, 0.04691048, 0.02879...
agyon+hallgat        [-0.019913174, 0.09712439, -0.061895028, 0.061...
agyon+vág            [-0.016664473, 0.07070805, -0.10135977, -0.005...
alul+marad           [-0.07809085, -0.03725612, 0.053516977, 0.1409...
alul+teljesít        [0.10239037, -0.1033645, 0.0638732, 0.01932127...
alább+hagy           [0.043724254, -0.13427377, 0.013410918, 0.1429...
alább+száll          [-0.06269403, -0.015194558, -0.0037674531, 0.0...
alá+ír

In [150]:
prev2 = mapping.fit_transform(invocab.vector.values[0])

ValueError: Expected 2D array, got 1D array instead:
array=[-0.03255332 -0.0296448   0.0669592   0.10458419  0.04360939 -0.08737686
  0.06697632  0.00259566  0.00058888 -0.09104761 -0.0469204  -0.02541944
 -0.10382952  0.02492477  0.08667573 -0.06522219  0.04900631 -0.04331347
 -0.04272046 -0.03471877  0.09307569 -0.01057233 -0.00839493 -0.02932172
  0.07085064 -0.07817982  0.03803245 -0.03071577 -0.00863978  0.07275739
  0.05276186 -0.01036315 -0.08687874  0.03310317 -0.07112531 -0.03645244
  0.03755444  0.0435865   0.03878628  0.06579787 -0.03843582  0.017313
  0.04176576  0.01028358  0.0018475  -0.07503915 -0.01317247  0.00088373
 -0.03392918  0.04006594 -0.00223854 -0.0155136   0.04253642 -0.00173141
  0.03992844  0.05682214 -0.03796554 -0.04595899  0.11066066 -0.04685317
 -0.08381969 -0.04119303  0.05893964  0.02326686 -0.02934791 -0.09824873
  0.06276221  0.09088541 -0.1416623   0.03976706  0.02530253  0.0570632
  0.0537821   0.0235364  -0.04433233 -0.09690356 -0.07745387  0.0239

Embedding $n$ connected components using meta-embedding (experimental):
> I suspect the spectral initialisation is failing for one reason or another.
This can often happen for particularly oddly distributed data. As a
workaround you can use ``init='random'`` as a parameter to UMAP. It should
stop the NaNs happening at least. This isn't ideal, but it should get you
past the immediate problem. I'll try to look into the deeper issue soon.

In [85]:
plt.scatter(prev2[:,0], prev2[:,1], c=prevlex['tokfreq'])

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [60]:
prevlex.loc['el+mond']['prevverb']

'elmond'